In [1]:
import datetime
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from functools import partial


latter_size = 14
plt.rcParams['legend.fontsize'] = latter_size 
plt.rcParams['font.size'] = latter_size 
plt.rcParams['axes.labelsize'] = latter_size
plt.rcParams['xtick.labelsize'] = latter_size
plt.rcParams['ytick.labelsize'] = latter_size

In [2]:
curr_dir = os.path.abspath(os.path.curdir)
os.chdir("/home/peregrinus/suntime")
import suntime.__main__ as suntime
os.chdir(curr_dir)

In [3]:
df = pd.read_pickle('./data/sj2_analise.pkl')
df_station = pd.read_pickle('./data/df_station_sort_re.pkl')

In [4]:
lat = df_station.loc[df_station['identificationstation'] == 'sj2']['latitude'].values[0]
long = df_station.loc[df_station['identificationstation'] == 'sj2']['longitude'].values[0]

In [5]:
def is_post_sunset_partial(date, lat, long):
    ns = 1e-9
    datetime_ref = datetime.datetime.utcfromtimestamp(date.astype(int) * ns)
    date_date = datetime_ref.date()
    date_time = datetime_ref.time()
    
    sun = suntime.Sun(lat, long)
    ss = sun.get_sunset_time(date_date)
    
    ss_time = ss.time()
    
    if date_time >= ss_time:
        return 1.0
    else:
        return 0.0
    
is_post_sunset = partial(is_post_sunset_partial, lat=lat, long=long)

def is_post_sunrise_partial(date, lat, long):
    ns = 1e-9
    datetime_ref = datetime.datetime.utcfromtimestamp(date.astype(int) * ns)
    date_date = datetime_ref.date()
    date_time = datetime_ref.time()
    
    sun = suntime.Sun(lat, long)
    sr = sun.get_sunrise_time(date_date)
    
    sr_time = sr.time()
    
    if date_time >= sr_time:
        return 1.0
    else:
        return 0.0
    
is_post_sunrise = partial(is_post_sunrise_partial, lat=lat, long=long)

In [6]:
state_post_sunset = []
for i in df.index.values:
    state_post_sunset.append(is_post_sunset(i))
state_post_sunset = np.array(state_post_sunset)
df['post_sunset'] = state_post_sunset

state_post_sunrise = []
for i in df.index.values:
    state_post_sunrise.append(is_post_sunrise(i))
state_post_sunrise = np.array(state_post_sunrise)
df['post_sunrise'] = state_post_sunrise

In [7]:
df

,vtec,vtec_dt,vtec_dt2,gvtec1,gvtec1_dt,gvtec2,gvtec2_dt,s4,post_sunset,post_sunrise
2013-12-01 03:30:00,33.624428,-0.428246,-0.020502,-3.216613,0.002355,1.826065,1.991653,0.163334,0.0,0.0
2013-12-01 03:40:00,33.205025,-0.546589,-0.118343,-3.122300,0.130198,4.588494,2.530373,0.165397,0.0,0.0
2013-12-01 03:50:00,32.539631,-0.772594,-0.226005,-2.810556,0.280756,7.786990,2.862830,0.168683,0.0,0.0
2013-12-01 04:00:00,31.469337,-1.082859,-0.310265,-2.320938,0.399753,10.972091,2.894375,0.173552,0.0,0.0
2013-12-01 04:10:00,29.995466,-1.429647,-0.346788,-1.764129,0.443303,13.734955,2.621609,0.178917,0.0,0.0
2013-12-01 04:20:00,28.194797,-1.748643,-0.318996,-1.279775,0.399478,15.846379,2.116655,0.183615,0.0,0.0
2013-12-01 04:30:00,26.140187,-1.970060,-0.221417,-0.971635,0.295367,17.265745,1.481490,0.186186,0.0,0.0
2013-12-01 04:40:00,23.954667,-2.040046,-0.069986,-0.856185,0.185794,18.030494,0.804197,0.186092,0.0,0.0
2013-12-01 04:50:00,21.846906,-1.938287,0.101759,-0.851142,0.127668,18.167486,0.137714,0.185635,0.0,0.0
2013-12-01 05:00:00,20.044188,-1.686945,0.251341,-0.811061,0.151654,17.689626,-0.491694,0.186621,0.0,0.0


In [8]:
df['gvtec1_dt_lag_9'] = df['gvtec1'] -  df['gvtec1'].shift(9)
df['gvtec2_dt_lag_20'] = df['gvtec2'] -  df['gvtec2'].shift(20)

df = df.dropna()
df.head()

,vtec,vtec_dt,vtec_dt2,gvtec1,gvtec1_dt,gvtec2,gvtec2_dt,s4,post_sunset,post_sunrise,gvtec1_dt_lag_9,gvtec2_dt_lag_20
2013-12-01 06:50:00,12.173075,-0.865222,-0.136979,-0.434405,-0.482498,8.790150,0.423989,0.178057,0.0,0.0,-0.249790,6.964085
2013-12-01 07:00:00,11.093473,-0.982551,-0.117329,-0.924772,-0.440270,9.082647,0.368625,0.182354,0.0,0.0,-1.295695,4.494153
2013-12-01 07:10:00,9.854814,-1.011528,-0.028977,-1.332911,-0.334433,9.263290,0.310082,0.185238,0.0,0.0,-2.245905,1.476301
2013-12-01 07:20:00,8.775170,-0.905486,0.106042,-1.563743,-0.173118,9.469463,0.296045,0.186712,0.0,0.0,-2.850710,-1.502628
2013-12-01 07:30:00,8.053244,-0.663704,0.241782,-1.547260,0.015597,9.792571,0.323477,0.187246,0.0,0.0,-2.949640,-3.942384


In [9]:
df.to_pickle('./data/sj2_analise_update.pkl')